## Preprocessing

In [6]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner as kt

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [8]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 156
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x <= cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 777
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x <= cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
# Convert categoricals
application_dummies = pd.get_dummies(application_df["APPLICATION_TYPE"])
class_dummies = pd.get_dummies(application_df["CLASSIFICATION"])
income_dummies = pd.get_dummies(application_df["INCOME_AMT"])
consider_dummies = pd.get_dummies(application_df["SPECIAL_CONSIDERATIONS"])
affiliation_dummies = pd.get_dummies(application_df["AFFILIATION"])
usecase_dummies = pd.get_dummies(application_df["USE_CASE"])
org_dummies = pd.get_dummies(application_df["ORGANIZATION"])

# Merge data into single df
application_df_1 = pd.concat([application_df, application_dummies, class_dummies, income_dummies, consider_dummies, affiliation_dummies, usecase_dummies, org_dummies], axis=1)

# Drop original / converted columns
application_df_1 = application_df_1.drop(columns=["APPLICATION_TYPE", "CLASSIFICATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS", "AFFILIATION", "USE_CASE", "ORGANIZATION"])

# Redundant columns
# "APPLICATION_TYPE" - "Other"
# "CLASSIFICATION" - "Other"
# "INCOME_AMT" - "0"
# "SPECIAL_CONSIDERATIONS" - "Y"
# "AFFILIATION" - "Other"
# "USE_CASE" - "Other"
# "ORGANIZATION" - "Corporation"
# Drop redundant columns
application_df_1 = application_df_1.drop(columns=["Other", "0", "Y", "Corporation"])

# Display
application_df_1.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,T10,T19,T3,T4,T5,T6,T7,...,Independent,National,Regional,CommunityServ,Heathcare,Preservation,ProductDev,Association,Co-operative,Trust
0,1,5000,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1


In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df_1["IS_SUCCESSFUL"].values
X = application_df_1.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=88)

<ipython-input-15-46a406eda181>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df_1.drop(["IS_SUCCESSFUL"],1).values


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [45]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 80)                2240      
                                                                 
 dense_11 (Dense)            (None, 30)                2430      
                                                                 
 dense_12 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4,701
Trainable params: 4,701
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5696 - accuracy: 0.7196
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7274
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7278
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7294
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7301
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7313
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7306
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7332
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5457 - accuracy: 0.7322
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accura

In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5576 - accuracy: 0.7335 - 690ms/epoch - 3ms/step
Loss: 0.5575553178787231, Accuracy: 0.7335277199745178


## Optimize Model #1 - Additional Data Preprocessing

In [49]:
# Take copy of original data with adjustment to 'APPLICATION_TYPE' & 'CLASSIFICATION'
application_df_2 = application_df.copy()
application_df_2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [50]:
# Review other variables value_counts 
# Look at AFFILIATION value counts for binning
application_df_2['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [51]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 64
affiliation_types_to_replace = application_df_2['AFFILIATION'].value_counts().loc[lambda x : x <= cutoff].index.tolist()

# Replace in dataframe
for aff in affiliation_types_to_replace:
    application_df_2['AFFILIATION'] = application_df_2['AFFILIATION'].replace(aff,"Other")

# Check to make sure binning was successful
application_df_2['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [52]:
# Look at USE_CASE value counts for binning
application_df_2['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [53]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 384
usecase_types_to_replace = application_df_2['USE_CASE'].value_counts().loc[lambda x : x <= cutoff].index.tolist()

# Replace in dataframe
for use in usecase_types_to_replace:
    application_df_2['USE_CASE'] = application_df_2['USE_CASE'].replace(use,"Other")

# Check to make sure binning was successful
application_df_2['USE_CASE'].value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [54]:
# Look at ORGANIZATION value counts for binning
application_df_2['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [55]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 486
org_types_to_replace = application_df_2['ORGANIZATION'].value_counts().loc[lambda x : x <= cutoff].index.tolist()

# Replace in dataframe
for org in org_types_to_replace:
    application_df_2['ORGANIZATION'] = application_df_2['ORGANIZATION'].replace(org,"Other")

# Check to make sure binning was successful
application_df_2['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [56]:
# Look at STATUS value counts for binning
application_df_2['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [57]:
# Look at STATUS value counts for binning
application_df_2['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [58]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
income_types_to_replace_1 = ['1-9999', '10000-24999']
income_types_to_replace_2 = ['5M-10M', '10M-50M', '50M+']

# Replace in dataframe
for inc in income_types_to_replace_1:
    application_df_2['INCOME_AMT'] = application_df_2['INCOME_AMT'].replace(inc,"1-24999")

for inc in income_types_to_replace_2:
    application_df_2['INCOME_AMT'] = application_df_2['INCOME_AMT'].replace(inc,"5M+")

# Check to make sure binning was successful
application_df_2['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1-24999           1271
1M-5M              955
5M+                564
Name: INCOME_AMT, dtype: int64

In [59]:
# Look at STATUS value counts for binning
application_df_2['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [60]:
# Display data
application_df_2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Other,1,1-24999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,1-24999,N,6692,1
4,T3,Independent,C1000,Other,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [61]:
# Convert categorical data to numeric with `pd.get_dummies`
# Convert categoricals
application_dummies = pd.get_dummies(application_df_2["APPLICATION_TYPE"])
class_dummies = pd.get_dummies(application_df_2["CLASSIFICATION"])
income_dummies = pd.get_dummies(application_df_2["INCOME_AMT"])
affiliation_dummies = pd.get_dummies(application_df_2["AFFILIATION"])
usecase_dummies = pd.get_dummies(application_df_2["USE_CASE"])
org_dummies = pd.get_dummies(application_df_2["ORGANIZATION"])
special_dummies = pd.get_dummies(application_df_2["SPECIAL_CONSIDERATIONS"])

# Merge data into single df
application_df_2 = pd.concat([application_df_2, application_dummies, class_dummies, income_dummies, affiliation_dummies, usecase_dummies, org_dummies, special_dummies], axis=1)

# Drop original / converted columns
application_df_2 = application_df_2.drop(columns=["APPLICATION_TYPE", "CLASSIFICATION", "INCOME_AMT", "AFFILIATION", "USE_CASE", "ORGANIZATION", "SPECIAL_CONSIDERATIONS"])

# Drop extremely imbalanced columns
#application_df_2 = application_df_2.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"])

# Redundant columns
# "APPLICATION_TYPE" - "Other"
# "CLASSIFICATION" - "Other"
# "INCOME_AMT" - "0"
# "SPECIAL_CONSIDERATIONS" - "Y"
# "AFFILIATION" - "Other"
# "USE_CASE" - "Other"
# "ORGANIZATION" - "Other"
# Drop redundant columns
application_df_2 = application_df_2.drop(columns=["Other", "0", "Y"])

# Display columns
application_df_2.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'T10', 'T19', 'T3', 'T4', 'T5',
       'T6', 'T7', 'T8', 'C1000', 'C1200', 'C2000', 'C2100', 'C3000',
       '1-24999', '100000-499999', '1M-5M', '25000-99999', '5M+',
       'CompanySponsored', 'Independent', 'Preservation', 'ProductDev',
       'Association', 'Trust', 'N'],
      dtype='object')

In [62]:
# Split our preprocessed data into our features and target arrays
y = application_df_2["IS_SUCCESSFUL"].values
X = application_df_2.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=88)

<ipython-input-62-d43c1d0b2bb7>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df_2.drop(["IS_SUCCESSFUL"],1).values


In [63]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 80)                2240      
                                                                 
 dense_14 (Dense)            (None, 30)                2430      
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4,701
Trainable params: 4,701
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [66]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5705 - accuracy: 0.7210
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5569 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7282
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7286
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7299
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7295
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7312
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7303
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accura

In [67]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6380 - accuracy: 0.7270 - 496ms/epoch - 2ms/step
Loss: 0.6380351185798645, Accuracy: 0.7269970774650574


## Optimize Model #2 - log(ASK_AMT)

In [68]:
# Make Copy of df_2
application_df_3 = application_df_2.copy()
application_df_3

,STATUS,ASK_AMT,IS_SUCCESSFUL,T10,T19,T3,T4,T5,T6,T7,...,1M-5M,25000-99999,5M+,CompanySponsored,Independent,Preservation,ProductDev,Association,Trust,N
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,1
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,1,0,1
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,1,0,1
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,1,0,1
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0,1
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,1,0,1


In [69]:
# Convert 'ASK_AMT' to 'LOG_ASK_AMT'
application_df_3["LOG_ASK_AMT"] = np.log2(application_df_3["ASK_AMT"])

# Drop original 'ASK_AMT' column
application_df_3 = application_df_3.drop(columns=["ASK_AMT"])

# Display
application_df_3.head()

,STATUS,IS_SUCCESSFUL,T10,T19,T3,T4,T5,T6,T7,T8,...,25000-99999,5M+,CompanySponsored,Independent,Preservation,ProductDev,Association,Trust,N,LOG_ASK_AMT
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,1,12.287712
1,1,1,0,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,16.728532
2,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,12.287712
3,1,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,12.708222
4,1,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,17.121513


In [70]:
# Split our preprocessed data into our features and target arrays
y = application_df_3["IS_SUCCESSFUL"].values
X = application_df_3.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=88)

<ipython-input-70-e13d6b6f59a5>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df_3.drop(["IS_SUCCESSFUL"],1).values


In [71]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [72]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 80)                2240      
                                                                 
 dense_17 (Dense)            (None, 30)                2430      
                                                                 
 dense_18 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4,701
Trainable params: 4,701
Non-trainable params: 0
_________________________________________________________________


In [73]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [74]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5709 - accuracy: 0.7191
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7256
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5532 - accuracy: 0.7271
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5504 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7308
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7307
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7308
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7319
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7324
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accura

In [75]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5639 - accuracy: 0.7336 - 520ms/epoch - 2ms/step
Loss: 0.5639458894729614, Accuracy: 0.7336443066596985


## Optimize Model #3 - Auto Optimize

In [38]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=50,
        max_value=100,
        step=10), activation=activation, input_dim=len(X_train[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 2, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=50,
            max_value=100,
            step=10),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [39]:
# Import the kerastuner library
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=25,
    hyperband_iterations=2)

In [40]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=25,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 16s]
val_accuracy: 0.7366763949394226

Best val_accuracy So Far: 0.7371428608894348
Total elapsed time: 00h 36m 09s


In [41]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 100, 'num_layers': 3, 'units_0': 100, 'units_1': 100, 'units_2': 70, 'units_3': 90, 'units_4': 100, 'tuner/epochs': 9, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 80, 'num_layers': 3, 'units_0': 80, 'units_1': 50, 'units_2': 90, 'units_3': 60, 'units_4': 100, 'tuner/epochs': 25, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 100, 'num_layers': 5, 'units_0': 80, 'units_1': 70, 'units_2': 50, 'units_3': 100, 'units_4': 80, 'tuner/epochs': 25, 'tuner/initial_epoch': 9, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}


In [42]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5521 - accuracy: 0.7371 - 656ms/epoch - 2ms/step
Loss: 0.5520650744438171, Accuracy: 0.7371428608894348
268/268 - 1s - loss: 0.5546 - accuracy: 0.7368 - 633ms/epoch - 2ms/step
Loss: 0.5545673966407776, Accuracy: 0.7367929816246033
268/268 - 1s - loss: 0.5513 - accuracy: 0.7367 - 683ms/epoch - 3ms/step
Loss: 0.5512571334838867, Accuracy: 0.7366763949394226


## Export Model

In [79]:
# Export our model to HDF5 file
model.save('AlphabetSoupCharity_Optimization.h5')